In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [82]:
dataset = pd.read_csv('data/train.csv')
dataset.head()
dataset.shape

(20800, 5)

In [83]:
nan_indexes = dataset.loc[pd.isna(dataset["text"]), :].index
dataset = dataset.drop(nan_indexes)
dataset.shape

(20761, 5)

In [84]:
X = dataset.text
y = dataset.label

In [86]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)